# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vaca_df = pd.read_csv('output_data/cities.csv')
vaca_df.head()

,City,Country,Lat,Lon,Temp,Humidity,Wind_Speed,Cloudiness,Date (UTC)
0,Mar Del Plata,AR,-38.00,-57.56,69.24,94.0,13.87,75.0,2020-03-04 22:22:19
1,New Norfolk,AU,-42.78,147.06,60.75,93.0,9.17,90.0,2020-03-04 22:22:20
2,Hobart,AU,-42.88,147.33,60.44,93.0,10.29,75.0,2020-03-04 22:20:18
3,Geraldton,AU,-28.77,114.60,64.40,63.0,6.93,33.0,2020-03-04 22:22:22
4,Albany,US,42.60,-73.97,41.79,42.0,13.87,40.0,2020-03-04 22:22:04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


humidity = vaca_df["Humidity"]

In [4]:
# Create a humidy heat map
fig = gmaps.figure(center=(0, 0), zoom_level=2)
locations = vaca_df[["Lat", "Lon"]]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions-

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* Drop any rows will null values.

In [5]:
#Ideal Conditions:
# Teperature between 70 and 80
# Cloudiness less than 15%
# Wind Speed less than 10 mph

ideal_df = vaca_df[(vaca_df['Temp'] >= 70) & (vaca_df['Temp'] <=80) & (vaca_df['Cloudiness'] <=15) & (vaca_df['Wind_Speed'] <=10)]
ideal_df = ideal_df.reset_index(drop=True)
ideal_df

,City,Country,Lat,Lon,Temp,Humidity,Wind_Speed,Cloudiness,Date (UTC)
0,Atar,MR,20.52,-13.05,78.40,7.0,7.09,0.0,2020-03-04 22:23:18
1,Sekoma,BW,-24.40,23.88,70.45,42.0,6.08,13.0,2020-03-04 22:23:21
2,Umm Lajj,SA,25.02,37.27,73.74,19.0,3.31,0.0,2020-03-04 22:23:29
3,Santa Vitoria,BR,-18.84,-50.12,77.90,80.0,6.22,3.0,2020-03-04 22:23:39
4,Castro,BR,-24.79,-50.01,70.45,63.0,0.51,0.0,2020-03-04 22:24:09
5,Pinole,US,38.00,-122.30,75.94,33.0,6.93,1.0,2020-03-04 22:23:26
6,Port Alfred,ZA,-33.59,26.89,71.01,83.0,5.99,0.0,2020-03-04 22:25:03
7,Werda,BW,-25.27,23.28,72.37,46.0,4.74,1.0,2020-03-04 22:22:03
8,Necochea,AR,-38.55,-58.74,76.10,69.0,4.00,0.0,2020-03-04 22:25:50
9,Akyab,MM,20.15,92.90,70.07,82.0,7.05,0.0,2020-03-04 22:25:53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel_df

hotel_df = ideal_df[["City", "Country", "Lat", "Lon"]]
hotel_df

,City,Country,Lat,Lon
0,Atar,MR,20.52,-13.05
1,Sekoma,BW,-24.40,23.88
2,Umm Lajj,SA,25.02,37.27
3,Santa Vitoria,BR,-18.84,-50.12
4,Castro,BR,-24.79,-50.01
5,Pinole,US,38.00,-122.30
6,Port Alfred,ZA,-33.59,26.89
7,Werda,BW,-25.27,23.28
8,Necochea,AR,-38.55,-58.74
9,Akyab,MM,20.15,92.90


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lon from hotel_df
    lat = row["Lat"]
    lon = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = (None)
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Atar.
Closest hotel is Bab Sahara.
------------
Retrieving Results for Index 1: Sekoma.


C:\Users\travi\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\travi\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
------------
Retrieving Results for Index 2: Umm Lajj.
Closest hotel is عروس الشاطئ للوحدات السكنية المفروشة.
------------
Retrieving Results for Index 3: Santa Vitoria.
Closest hotel is HOTEL GUIMA.
------------
Retrieving Results for Index 4: Castro.
Closest hotel is Pousada Rota Dos Tropeiros.
------------
Retrieving Results for Index 5: Pinole.
Closest hotel is Holiday Inn Express & Suites San Pablo - Richmond Area.
------------
Retrieving Results for Index 6: Port Alfred.
Closest hotel is The Halyards Hotel.
------------
Retrieving Results for Index 7: Werda.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Necochea.
Closest hotel is Casa del Parque.
------------
Retrieving Results for Index 9: Akyab.
Closest hotel is Hotel Memory.
------------
Retrieving Results for Index 10: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 11: Raga.
Missing field/result... skipping.
------------
Ret

In [11]:
#clean hotel data(drop na)
#drop "na" values
hotel_df = hotel_df.dropna(how='any', axis=0)

hotel_df.head(10)


,City,Country,Lat,Lon,Hotel Name
0,Atar,MR,20.52,-13.05,Bab Sahara
2,Umm Lajj,SA,25.02,37.27,عروس الشاطئ للوحدات السكنية المفروشة
3,Santa Vitoria,BR,-18.84,-50.12,HOTEL GUIMA
4,Castro,BR,-24.79,-50.01,Pousada Rota Dos Tropeiros
5,Pinole,US,38.00,-122.30,Holiday Inn Express & Suites San Pablo - Richm...
6,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
8,Necochea,AR,-38.55,-58.74,Casa del Parque
9,Akyab,MM,20.15,92.90,Hotel Memory
12,Laguna,US,38.42,-121.42,Hilton Garden Inn Sacramento Elk Grove
14,Pahrump,US,36.21,-115.98,Saddle West Hotel Casino RV Resort


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Bab Sahara</dd>\n<dt>City</dt><dd>Atar</dd>\n<dt>Country</dt><dd>MR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>عروس الشاطئ للوحدات السكنية المفروشة</dd>\n<dt>City</dt><dd>Umm Lajj</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>HOTEL GUIMA</dd>\n<dt>City</dt><dd>Santa Vitoria</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Pousada Rota Dos Tropeiros</dd>\n<dt>City</dt><dd>Castro</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Holiday Inn Express & Suites San Pablo - Richmond Area</dd>\n<dt>City</dt><dd>Pinole</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>The Halyards Hotel</dd>\n<dt>City</dt><dd>Port Alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Casa del Parque</dd>\n<dt>City</dt><dd>Necochea</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Memory</dd>\n<dt>City</dt><dd>Akyab</dd>\n<dt>Country</dt><dd>

In [13]:
locations = hotel_df[["Lat", "Lon"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))